<a href="https://colab.research.google.com/github/byuccl/digital_design_colab/blob/dev/Lab4_Stopwatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="right">
<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab/main/Dataflow/media/BYU.svg"
width="500" height="" style="display: block; margin: 0 auto" />
</p>


<h1>
<summary>
<h1 align="center">
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<b>$\color{DarkBlue}{\underline{\textbf{Stopwatch}}}$<bol><p align="right">
</summary>
</h1>

## **Setup**
Click the Play Button. No need to expand. Do not touch this. 
<details><summary>Packages to be downloaded</summary>
Things it needs to install:  <br>

 *    Verilator- the simulator  <br>
 *    PyGithub - Helps with simulation    <br>
 *    VCD - Generates waveforms from `.vcd` files  <br>
 *    Widgets - Each widget must first be generated then will be loaded when played
</details>

In [ ]:
#@title Setup
!git clone https://github.com/yne/vcd.git
!make -C /content/vcd
%cd /content/vcd
!make install
!apt-get install verilator >/dev/null
%cd /
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
import requests

!mkdir -p /content/tmp_code
#Creates a text document using the Raw github url. This text document will be used as a .py file for imports
def import_text(text):
  url = "https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Labs/stopwatch_lab/files/%s" % text
  resp = requests.get(url)
  with open(text, 'wb') as f:
    f.write(resp.content)

import_text("import_all.py")
from import_all import *
import_source()
import_packages()
from simulation import *
from convertData import *
!pip install --upgrade git+https://github.com/anon36424/nb_js_diagrammers.git
%load_ext nb_js_diagrammers


# Stopwatch

###Frequency Vs Period

Frequency is the number of times something occurs in a given time frame, while period is the amount of time it takes for something to occur. They are inverses of one another. $Period = \frac{1}{Frequency}$ and $Frequency = \frac{1}{Period}$. 

This is important when dealing with Clocks in circuits.

Example:  
You are running around a pillar while 
playing tag. 

You run around it 3 times every minute. That is your frequency.  

The period would be $\frac{1}{Frequency}$, so every 1/3 of a minute or 20 seconds you would complete a circle. This is the period.


So why does Frequency and Period matter?    

Timing is very important because of the FPGAs built in clock. The clock keeps all elements in the circuit working together and prevents conflicts.    

In this lab, inorder to create a stopwatch, you will need to know how many clock cycles occur in a given time frame, then after counting that many, increment the stopwatch.

In [ ]:
#@title
frq_1[0,2].on_click(check_frq_1)
header_button = create_expanded_button('If the Period is 5 Microseconds what is the Frequency?', 'primary', '600px')
widgets.VBox([widgets.AppLayout(header=header_button,footer=None),frq_1])




In [ ]:
#@title
frq_2[0,2].on_click(check_frq_2)
header_button = create_expanded_button('If the Period is 25 ns what is the Frequency? ', 'primary', '600px')
widgets.VBox([widgets.AppLayout(header=header_button,footer=None),frq_2])



In [ ]:
#@title
frq_3[0,2].on_click(check_frq_3)
header_button = create_expanded_button('If the Frequency is 10GHZ what is the Period?', 'primary', '600px')
widgets.VBox([widgets.AppLayout(header=header_button,footer=None),frq_3])

In [ ]:
#@title
frq_4[0,2].on_click(check_frq_4)
header_button = create_expanded_button('If the frequency is 25Hz what is the Period?', 'primary', '600px')
widgets.VBox([widgets.AppLayout(header=header_button,footer=None),frq_4])

### The Counter Module

<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab/main/Stopwatch/media/mod_counter.png"
width="400" height="150" style="display: block; margin: 0 auto " />
<br>

| Module Name: | mod_counter|||
| ---------- | ----------- |--|--|
| Port Name      | Direction       |Width|Function|
|reset 	|Input 	|1| Active high reset|
|clk 	|Input 	|1| Clock used for timing|
|increment 	|Input 	|1 | should only increment when enable if its high|
| MOD_VALUE|	Parameter| 	N/A| Value at which counter should reset, default of 10 	|
| rollover	|Output 	|1| 	High for one cycle when counter rolls over|
|count| 	Output 	|4| 	Current value of the counter|


<br>

When increment is high, the counter should begin increasing. When it reaches the MOD_VALUE, it should reset to zero and rollover should be high for that clock cycle. 



In [ ]:
#@title Launch Simulation Workspace
createSimulationWorkSpace("tmp_code/mod_counter")


### The Stopwatch Module



<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab/main/Stopwatch/media/stopwatch_module.jpg"
width="500" style="display: block; margin: 0 auto " />
<br><br>

| Module Name: | stopwatch |||
| -- --------- | ----------- |--|--|
| Port Name      | Direction       |Width|Function|
|clk 	|Input 	|1| The Clock Signal|
|reset 	|Input 	|1| Active high reset|  
|run 	|Input 	|1 | High when timer should be running|
| digit0	|Output 	|4| The value of the hundredths of a second digit|
| digit1	|Output 	|4| The value of the tenths of a second digit|
| digit2	|Output 	|4| The value of the seconds digit|
| digit3	|Output 	|4| The value of the tens of seconds digit|

<br>


You will  have to create a separate counter. This one will reset and give a rollover value every .01s. This cannot be an instantiation of `mod_counter` because its `count` value will be wider than 4 bits. This `rollover` will be fed into `digit0` as the `increment`.


You will also have to instantiate 4 different instances of `mod_counter`, `digit0` through `digit3`.

When `digit0` reaches 9 it should increment `digit1` and go back to `0`. So on for all 4 digits. `digit3` should rollver at `5` (60 seconds in a minute after all)

In [ ]:
#@title Launch Simulation Workspace
createSimulationWorkSpace("tmp_code/stopwatch")


## Creating a Top Module

# Top Level Module
| Module Name: | stopwatch_top|||  
| ----------- | ----------- |--|--|
| Port Name      | Direction |Width|Function|
|clk 	|Input 	|1| The Clock Signal |
| btnc 	|Input 	|1 | Active High Reset|
| sw	|Input 	|1| High when stopwatch should be running |
| anode	|Output 	|4| Seven-segment anode values, from Seven-Segment Controller|
| segment	|Output 	|8| Seven-segment segment values,  from Seven-Segment Controller|  

The `clk` signal is defined in the XDC.  

When `btnc` is pressed, the stopwatch should go to `0` for all digits.

While the switch is high, the stopwatch should begin to increment. You should time it with a different stopwatch to make sure that your stopwatch is keeping accurate time.  

You can concatenate the 4 digits together to create the `datain` signal for `SevenSegmentControl`.



NOTE: We are giving you the Seven Segement Control Module. This allows you to show 4 unique digits by quickly alternativing between the 4 different valuse faster than your eyes can see.   

* `dataIn`: 16 bits, each 4 bits are a hex value which is shown as a digit.  
* `digitDisplay`: 4 bits, if you want all 4 digits, pass `1111` into it.  
* `digitPoint`: This determines which digit point is on.   
* `anode`: Hook this up to the output `anode` signal 
* `segment`: Hook up to the output `segment` signal 
* `reset`, and `clk`: Self-explanatory inputs

In [ ]:
#@title Launch Simulation Workspace
createSimulationWorkSpace("tmp_code/stopwatch_top")


## Compiling with the F4PGA Toolchain


### Installing the Toolchain

In [ ]:
!apt install -y git wget xz-utils

In [ ]:
%%bash
cd /content
git clone https://github.com/chipsalliance/f4pga-examples
cd f4pga-examples
wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O conda_installer.sh


In [ ]:
%%bash
cd /content/f4pga-examples
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
bash conda_installer.sh -u -b -p $F4PGA_INSTALL_DIR/$FPGA_FAM/conda;
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
conda env create -f $FPGA_FAM/environment.yml

In [ ]:
#Download F4PGA Arch Defs
%%bash
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
conda activate xc7
mkdir -p $F4PGA_INSTALL_DIR/xc7/install

F4PGA_TIMESTAMP='20220907-210059'
F4PGA_HASH='66a976d'
export F4PGA_PACKAGES='install-xc7 xc7a50t_test'

for PKG in $F4PGA_PACKAGES; do
  wget -qO- https://storage.googleapis.com/symbiflow-arch-defs/artifacts/prod/foss-fpga-tools/symbiflow-arch-defs/continuous/install/${F4PGA_TIMESTAMP}/symbiflow-arch-defs-${PKG}-${F4PGA_HASH}.tar.xz | tar -xJC $F4PGA_INSTALL_DIR/${FPGA_FAM}
done

In [ ]:
#@title Generate an XDC File
%%bash -c 'cat > /content/tmp_code/xdc.xdc'
###################################################################
###################################################################
set_property -dict { PACKAGE_PIN W5   IOSTANDARD LVCMOS33 } [get_ports clk]
create_clock -add -name sys_clk_pin -period 10.00 -waveform {0 5} [get_ports clk]

## Buttons

set_property -dict { PACKAGE_PIN U18   IOSTANDARD LVCMOS33 } [get_ports { btnc }];
# set_property -dict { PACKAGE_PIN T18   IOSTANDARD LVCMOS33 } [get_ports { btnu }];
# set_property -dict { PACKAGE_PIN W19   IOSTANDARD LVCMOS33 } [get_ports { btnl }];
# set_property -dict { PACKAGE_PIN T17   IOSTANDARD LVCMOS33 } [get_ports { btnr }];
# set_property -dict { PACKAGE_PIN U17   IOSTANDARD LVCMOS33 } [get_ports { btnd }];

## Switches
set_property -dict { PACKAGE_PIN V17   IOSTANDARD LVCMOS33 } [get_ports { sw }];
# set_property -dict { PACKAGE_PIN V16   IOSTANDARD LVCMOS33 } [get_ports { sw[1] }];
# set_property -dict { PACKAGE_PIN W16   IOSTANDARD LVCMOS33 } [get_ports { sw[2] }];
# set_property -dict { PACKAGE_PIN W17   IOSTANDARD LVCMOS33 } [get_ports { sw[3] }];
# set_property -dict { PACKAGE_PIN W15   IOSTANDARD LVCMOS33 } [get_ports { sw[4] }];
# set_property -dict { PACKAGE_PIN V15   IOSTANDARD LVCMOS33 } [get_ports { sw[5] }];
# set_property -dict { PACKAGE_PIN W14   IOSTANDARD LVCMOS33 } [get_ports { sw[6] }];
# set_property -dict { PACKAGE_PIN W13   IOSTANDARD LVCMOS33 } [get_ports { sw[7] }];
# set_property -dict { PACKAGE_PIN V2    IOSTANDARD LVCMOS33 } [get_ports { sw[8] }];
# set_property -dict { PACKAGE_PIN T3    IOSTANDARD LVCMOS33 } [get_ports { sw[9] }];
# set_property -dict { PACKAGE_PIN T2    IOSTANDARD LVCMOS33 } [get_ports { sw[10] }];
# set_property -dict { PACKAGE_PIN R3    IOSTANDARD LVCMOS33 } [get_ports { sw[11] }];
# set_property -dict { PACKAGE_PIN W2    IOSTANDARD LVCMOS33 } [get_ports { sw[12] }];
# set_property -dict { PACKAGE_PIN U1    IOSTANDARD LVCMOS33 } [get_ports { sw[13] }];
# set_property -dict { PACKAGE_PIN T1    IOSTANDARD LVCMOS33 } [get_ports { sw[14] }];
# set_property -dict { PACKAGE_PIN R2    IOSTANDARD LVCMOS33 } [get_ports { sw[15] }];
##7 Segment Display
set_property -dict { PACKAGE_PIN W7   IOSTANDARD LVCMOS33 } [get_ports {segment[0]}]
set_property -dict { PACKAGE_PIN W6   IOSTANDARD LVCMOS33 } [get_ports {segment[1]}]
set_property -dict { PACKAGE_PIN U8   IOSTANDARD LVCMOS33 } [get_ports {segment[2]}]
set_property -dict { PACKAGE_PIN V8   IOSTANDARD LVCMOS33 } [get_ports {segment[3]}]
set_property -dict { PACKAGE_PIN U5   IOSTANDARD LVCMOS33 } [get_ports {segment[4]}]
set_property -dict { PACKAGE_PIN V5   IOSTANDARD LVCMOS33 } [get_ports {segment[5]}]
set_property -dict { PACKAGE_PIN U7   IOSTANDARD LVCMOS33 } [get_ports {segment[6]}]

#Anodes 

set_property -dict { PACKAGE_PIN U2   IOSTANDARD LVCMOS33 } [get_ports {anode[0]}]
set_property -dict { PACKAGE_PIN U4   IOSTANDARD LVCMOS33 } [get_ports {anode[1]}]
set_property -dict { PACKAGE_PIN V4   IOSTANDARD LVCMOS33 } [get_ports {anode[2]}]
set_property -dict { PACKAGE_PIN W4   IOSTANDARD LVCMOS33 } [get_ports {anode[3]}]
##

### Compiling with the Toolchain

In [ ]:
#This creates the Makefile
with open("/content/tmp_code/Makefile", "w") as f:
  f.write("""current_dir := ${CURDIR}
TARGET := basys3

TOP := stopwatch_top

XDC := ${current_dir}/*.xdc

SOURCES := $(wildcard ${current_dir}/*.v ${current_dir}/*.sv)

include /content/f4pga-examples/common/common.mk
""")

In [ ]:
#Make the project
%%bash
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
export TARGET=basys3
export FOLDER=Stopwatch
export FILES="Makefile xdc.xdc stopwatch_top.sv mod_counter.sv"
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
mkdir /content/"$FOLDER"
cd /content/tmp_code
for file in $FILES
do 
    mv $file /content/"$FOLDER"/$file
done
conda activate xc7
cd /content/$FOLDER
time make 2> ./error.log 1>./compile.log
if [ -f "/content/$FOLDER/build/basys3/*.bit" ]; then
    cp /content/$FOLDER/build/basys3/*.bit /content/$FOLDER.bit
else
    echo "An Error has occurred and the bit stream has not been generated:"
    cat ./compile.log
    cat ./error.log
fi

## Testing it on the board

You can use this configuration file and the program openOCD to download the bitstream to your board. You will need to download this file and the bitstream to your local machine.

In [ ]:
#@title Create File for openocd
folder = "Stopwatch"

with open("/content/{folder}.cfg", "w") as f:
  f.write("""interface ftdi
ftdi_device_desc "Digilent USB Device"
ftdi_vid_pid 0x0403 0x6010
# channel 1 does not have any functionality
ftdi_channel 0
# just TCK TDI TDO TMS, no reset
ftdi_layout_init 0x0088 0x008b
reset_config none
adapter_khz 10000

source [find cpld/xilinx-xc7.cfg]
source [find cpld/jtagspi.cfg]
init

puts [irscan xc7.tap 0x09]

set xc7a35t "0362D093"
set xc7a100t "13631093"
set code [drscan xc7.tap 32 0]  
puts $code

if { $code == $xc7a35t} {
    puts "The board has an xc7a35t"
}

if { $code == $xc7a100t} {
    puts "The board has an xc7a100t"
}

puts "Programming..."
"""
+
f"""
pld load 0 {folder}.bit
exit"""
)

In [ ]:
#@title Create Zip
%cd /content
!export FOLDER=Stopwatch
!mv $FOLDER/build/basys3/*.bit ./
!zip $FOLDER.zip *.bit $FOLDER.cfg